In [31]:
from dotenv import load_dotenv
load_dotenv()

# Use the variable with:
import os
API_KEY = os.getenv("API_KEY")
STEAM_ID = os.getenv("STEAM_ID")

import requests
import pandas as pd
from tqdm.auto import tqdm

General flow:
- Start with my Steam ID.
- Get a list of my games and save it in a dataframe.
- Everytime the dataframe is updated, save it in a csv file.
- Append the SteamID to the list of already visited SteamIDs.
- For each SteamID, get a list of their friends, check if they are already in the list of visited SteamIDs, and if not, add them to the list of SteamIDs to visit.
- Save the list of SteamIDs to visit in a csv file.
- Repeat the process.

In [32]:
# Takes a Steam ID and returns a DataFrame row with all of the user's games and playtime
def get_games(steam_id):
    url = f'http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/?key={API_KEY}&steamid={steam_id}&format=json&include_played_free_games=1'
    response = requests.get(url)
    data = response.json()
    data = data['response']
    # only extract app_id and playtime_forever from data['games']
    clean_data = []
    if 'games' not in data:
        return None
    for game in data['games']:
        if game['playtime_forever'] == 0:
            continue
        clean_data.append({'id': game['appid'], 'time': game['playtime_forever']})
    games = pd.DataFrame({'Total Games': data['game_count'], 'Played Games': len(clean_data), 'Games': [clean_data]}, index=[steam_id])
    return games

In [33]:
# Takes a Steam ID and returns a list of the user's friends
def get_friends(steam_id, visited_friends):
    url = f'http://api.steampowered.com/ISteamUser/GetFriendList/v0001/?key={API_KEY}&steamid={steam_id}&relationship=friend'
    response = requests.get(url)
    data = response.json()
    if 'friendslist' not in data:
        return None
    friends_to_visit = []
    for friend in data['friendslist']['friends']:
        if friend['steamid'] not in visited_friends:
            friends_to_visit.append(friend['steamid'])
    return friends_to_visit

In [34]:
# Saves all already visited friend IDs and saves it to a file
def save_visited_friends(visited_friends):
    with open('visited_friends.txt', 'w') as f:
        for friend in visited_friends:
            f.write(f'{friend}\n')

# Saves the queue of friends to visit to a file
def save_friends_queue(friends_queue):
    with open('friends_queue.txt', 'w') as f:
        for friend in friends_queue:
            f.write(f'{friend}\n')

# Appends the games dataframe to a csv file (or creates a new one if it doesn't exist)
def save_games(games, filename, overwrite=False):
    if not os.path.isfile(filename) or overwrite:
        games.to_csv(filename, header=['Total Games', 'Played Games', 'Games'], index_label='Steam ID')
    else:
        games.to_csv(filename, mode='a', header=False)

In [35]:
overwrite = True # Set to True to start from scratch

# Read queue.txt if exists, otherwise start with STEAM_ID
if os.path.exists('friends_queue.txt') and not overwrite:
    with open('friends_queue.txt', 'r') as f:
        friends_queue = f.read().splitlines()
else:
    friends_queue = [STEAM_ID]

# Read visited_friends.txt if exists
if os.path.exists('visited_friends.txt') and not overwrite:
    with open('visited_friends.txt', 'r') as f:
        visited_friends = f.read().splitlines()
else:
    visited_friends = []

if overwrite:
    if os.path.exists('user_games_data.csv'):
        os.remove('user_games_data.csv')

iterations = 10

for i in tqdm(range(iterations)):
    user = friends_queue.pop(0)
    visited_friends.append(user)

    if (i+1)%100 == 0:
        print(f"Queue: {len(friends_queue)} | Visited total: {len(visited_friends)}")

    # Get games and append them to the csv
    games = get_games(user)

    
    # Get (not yet visited) friends of the user and append them to the queue
    friends = get_friends(user, visited_friends)
    if friends is not None and len(friends) + len(friends_queue) < 3000:
        friends_queue.extend(friends)

    if games is not None and games['Played Games'][0] > 0:
        save_games(games, 'user_games_data.csv')

    # Save the visited friends and the friends queue
    save_visited_friends(visited_friends)
    save_friends_queue(friends_queue)
    

  0%|          | 0/10 [00:00<?, ?it/s]

In [27]:
games = pd.read_csv('user_games_data.csv', index_col=0)

In [28]:
games.loc[games['Total Games'] < 50]

,Total Games,Played Games,Games
Steam ID,,,
76561197978985924,22,8,"[{'id': 10, 'time': 20007}, {'id': 240, 'time'..."


In [30]:
games

,Total Games,Played Games,Games
Steam ID,,,
76561198280830500,1154,882,"[{'id': 4000, 'time': 77748}, {'id': 400, 'tim..."
76561197962287503,5613,3815,"[{'id': 10, 'time': 3}, {'id': 70, 'time': 77}..."
76561197977849691,11931,7129,"[{'id': 10, 'time': 1}, {'id': 220, 'time': 26..."
76561197978985924,22,8,"[{'id': 10, 'time': 20007}, {'id': 240, 'time'..."
76561197982447568,956,689,"[{'id': 20, 'time': 330}, {'id': 50, 'time': 3..."
76561197986480371,2544,316,"[{'id': 10, 'time': 56}, {'id': 240, 'time': 2..."
76561197991669382,11897,2235,"[{'id': 10, 'time': 5}, {'id': 240, 'time': 11..."
76561197997628919,332,156,"[{'id': 240, 'time': 4418}, {'id': 4000, 'time..."
76561198004311443,827,648,"[{'id': 10, 'time': 2628}, {'id': 80, 'time': ..."


GetOwnedGames - include_played_free_games
https://developer.valvesoftware.com/wiki/Steam_Web_API#GetPlayerSummaries_.28v0001.29

data przeczyścić bo zajmuje w cholerę miejsca